# Parquet konvertering-, partisjonering- og filtreringsverktøy for AIS-data

Dette verktøyet konverterer Parquet-filer til GeoParquet-format, partisjonerer på angitte kolonner og filtrerer på angitt dato og radius rundt Kristiansand sentrum. Data vises på tabell og kart.

Verktøyet er en løsningsmodell som bruker GeoPandas for følgende brukerhistorie:

"Som havnesjef i Kristiansand kommune ønsker jeg å vite hvilke skip som befant seg innen 6 km fra Kristiansand sentrum på en bestemt dato."

### Hvordan bruke dette verktøyet
1. Last opp dine geografiske datafiler til mappen `data/raw`
2. Kjør alle cellene i denne notebooken
3. De konverterte filene vil bli lagret i mappen `data/processed`

### Støttede filformater

- Parquet-filer med geografisk informasjon

In [6]:
# Global variabel for mappestier
_current_observer = None
global_paths = None

# Sett standardverdier for parameterne
data_dir = "./data"
num_files = 5

In [7]:
import argparse
import pandas as pd
import geopandas as gpd
from shapely import wkt

from shapely import wkb
from shapely.geometry import Point, Polygon
import time
import shutil
from datetime import datetime
import os
import datetime

import glob
import random

import folium
from folium import plugins
from shapely.geometry import Point, Polygon

from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

from shapely.geometry import Point
import pyproj
from shapely.ops import transform
from functools import partial

def find_newest_folder(folder_path, use_foldername=True):
    """
    Finner den nyeste mappen i en mappe basert på mappenavnet (dato) eller modifiseringstidspunkt.
    Leter etter mappenavn på formatet: hais_2024-12-24.snappy.parquet

    Args:
        folder_path (str): Stien til mappen hvor undermappene skal søkes
        use_foldername (bool): Om dato i mappenavn skal brukes (True) eller sist endret tid (False)

    Returns:
        tuple: (str: Stien til den nyeste mappen, str: Datoen som string i format 'YYYY-MM-DD',
               eller None hvis det ikke finnes mapper eller dato ikke ble funnet)
    """
    try:
        # Sjekk om mappen eksisterer
        if not os.path.exists(folder_path):
            print(f"Mappen {folder_path} eksisterer ikke")
            return None, None

        # Finn alle mapper i mappen (ikke filer)
        folders = [os.path.join(folder_path, f) for f in os.listdir(folder_path)
                  if os.path.isdir(os.path.join(folder_path, f))]

        if not folders:
            print(f"Ingen mapper funnet i {folder_path}")
            return None, None

        if use_foldername:
            # Finn den nyeste mappen basert på dato i mappenavnet
            newest_date = None
            newest_folder = None
            date_string = None

            for folder in folders:
                foldername = os.path.basename(folder)
                # Søk etter dato i formatet YYYY-MM-DD i mappenavnet
                date_match = re.search(r'(\d{4}-\d{2}-\d{2})', foldername)

                if date_match:
                    date_str = date_match.group(1)
                    try:
                        folder_date = datetime.datetime.strptime(date_str, '%Y-%m-%d').date()

                        if newest_date is None or folder_date > newest_date:
                            newest_date = folder_date
                            newest_folder = folder
                            date_string = date_str
                    except ValueError:
                        # Hvis datoen ikke kan tolkes, hopp over denne mappen
                        continue

            if newest_folder:
                return newest_folder, date_string
            else:
                print("Ingen mapper med gyldig datoformat funnet. Bruker modifiseringstidspunkt i stedet.")
                # Fall tilbake til modifiseringstidspunkt
                newest_folder = max(folders, key=os.path.getmtime)
                # Ingen dato funnet i mappenavn, så vi returnerer None som date_string
                return newest_folder, None
        else:
            # Finn den nyeste mappen basert på modifiseringstidspunkt
            newest_folder = max(folders, key=os.path.getmtime)

            # Sjekk om vi kan finne en dato i mappenavnet til den nyeste mappen
            foldername = os.path.basename(newest_folder)
            date_match = re.search(r'(\d{4}-\d{2}-\d{2})', foldername)
            date_string = date_match.group(1) if date_match else None

            return newest_folder, date_string

    except Exception as e:
        print(f"Feil ved søk etter nyeste mappe: {str(e)}")
        return None, None
    
def create_folders(data_folder="./data"):
    """
    Oppretter nødvendig mappestruktur for konvertering.

    Args:
        data_folder: Sti til hovedmappen for data

    Returns:
        dict: Stier til opprettede mapper
    """
    # Definer mappestruktur
    raw_folder = os.path.join(data_folder, "raw")
    processed_folder = os.path.join(data_folder, "processed")
    source_folder = os.path.join(data_folder, "source")
    incoming_folder = os.path.join(data_folder, "incoming")

    # Opprett mapper
    os.makedirs(data_folder, exist_ok=True)
    os.makedirs(raw_folder, exist_ok=True)
    os.makedirs(processed_folder, exist_ok=True)
    os.makedirs(source_folder, exist_ok=True)
    os.makedirs(incoming_folder, exist_ok=True)

    # Returner stier for senere bruk
    return {
        "data_folder": os.path.abspath(data_folder),
        "raw_folder": os.path.abspath(raw_folder),
        "processed_folder": os.path.abspath(processed_folder),
        "source_folder": os.path.abspath(source_folder),
        "incoming_folder": os.path.abspath(incoming_folder)
    }

def convert_parquet_to_geoparquet(file_path, output_path, partition_columns=None):
    """
    Konverterer parquet-fil til GeoParquet-format med partisjonering.
    Args:
        file_path: Sti til parquet-filen
        output_path: Sti hvor GeoParquet-filen skal lagres
        partition_columns: Liste av kolonnenavn som skal brukes for partisjonering
    """
    try:
        df = pd.read_parquet(file_path)
    except Exception as e:
        print(f"Kunne ikke lese parquet-fil: {e}")
        return False

    # Håndter tidspartisjonering
    time_partitioning = False
    if partition_columns and 'date_time_utc' in partition_columns:
        partition_columns.remove('date_time_utc')
        time_partitioning = True

    # Legg til alle tidspartisjoneringskolonner, men vi vil bare partisjonere på time
    if time_partitioning:
        df = add_time_partitioning_columns(df)
        # Sett opp partisjonering kun på 'hour'
        partition_columns = ['hour'] + (partition_columns or [])

    # Opprett GeoDataFrame
    gdf = create_geodataframe(df)
    if gdf is None:
        return False

    # Sjekk at alle partisjoneringskolonner finnes
    if partition_columns and not all(col in gdf.columns for col in partition_columns):
        print(f"Advarsel: Ikke alle partisjoneringskolonner finnes i datasettet")
        missing = [col for col in partition_columns if col not in gdf.columns]
        print(f"Manglende kolonner: {missing}")
        return False

    # Lagre med partisjonering (bare time)
    if partition_columns:
        return save_partitioned_geoparquet(gdf, output_path, partition_columns)
    else:
        gdf.to_parquet(output_path)
        return output_path
    
def add_time_partitioning_columns(df, time_column='date_time_utc'):
    """
    Legger til kolonner for tidspartisjonering (kun time for partisjonering, men beholder dato-kolonner).
    """
    if time_column not in df.columns:
        print(f"Advarsel: Tidsstempelkolonne '{time_column}' finnes ikke")
        return df

    if not pd.api.types.is_datetime64_any_dtype(df[time_column]):
        print(f"Advarsel: Kolonnen '{time_column}' er ikke en datetime-kolonne")
        return df

    df_with_time = df.copy()
    df_with_time['year'] = df_with_time[time_column].dt.year
    df_with_time['month'] = df_with_time[time_column].dt.month
    df_with_time['day'] = df_with_time[time_column].dt.day
    df_with_time['hour'] = df_with_time[time_column].dt.hour

    return df_with_time

def create_geodataframe(df):
    """
    Oppretter en GeoDataFrame fra en DataFrame ved å finne koordinater eller geometrikolonner.
    """

    # Sjekk for lat/long kolonner
    if 'longitude' in df.columns and 'latitude' in df.columns:
        try:
            return gpd.GeoDataFrame(
                df,
                geometry=gpd.points_from_xy(df.longitude, df.latitude),
                crs="EPSG:4326"
            )
        except Exception as e:
            print(f"Kunne ikke opprette geometri fra lat/long: {e}")

    # Sjekk for andre geometrikolonner
    geom_columns = [col for col in df.columns if any(
        term in col.lower() for term in ['geom', 'coord', 'point', 'polygon', 'linestring', 'wkt']
    )]

    for col in geom_columns:
        if df[col].dtype != 'object':
            continue

        try:
            geom = df[col].apply(wkt.loads)
            return gpd.GeoDataFrame(df, geometry=geom, crs="EPSG:4326")
        except Exception:
            continue

    return None

def save_partitioned_geoparquet(gdf, output_path, partition_columns):
    """
    Lagrer en GeoDataFrame som partisjonert GeoParquet.
    """

    # Konverterer geometri til WKB for å kunne partisjonere
    df_med_wkb = gdf.copy()
    df_med_wkb['geometry_wkb'] = df_med_wkb['geometry'].apply(lambda geom: wkb.dumps(geom))
    df_for_partisjon = df_med_wkb.drop(columns=['geometry'])

    # Utfør partisjonering
    df_for_partisjon.to_parquet(output_path, partition_cols=partition_columns)

    # Konverter hver partisjonert fil tilbake til GeoParquet
    convert_partitioned_files_to_geoparquet(output_path)
    return output_path

def convert_partitioned_files_to_geoparquet(root_folder):
    """
    Konverterer alle partisjonerte parquet-filer til GeoParquet format.
    """
    error_count = 0

    for root, _, files in os.walk(root_folder):
        for file in files:
            if not file.endswith('.parquet'):
                continue

            parquet_path = os.path.join(root, file)
            try:
                # Les dataframe
                part_df = pd.read_parquet(parquet_path)

                # Hopp over hvis den ikke har geometry_wkb
                if 'geometry_wkb' not in part_df.columns:
                    continue

                # Konverter WKB tilbake til geometri
                part_df['geometry'] = part_df['geometry_wkb'].apply(lambda x: wkb.loads(x))
                part_df = part_df.drop(columns=['geometry_wkb'])

                # Lag GeoDataFrame
                part_gdf = gpd.GeoDataFrame(part_df, geometry='geometry', crs="EPSG:4326")

                # Skriv GeoParquet-filen
                part_gdf.to_parquet(parquet_path)
            except Exception as e:
                print(f"Feil ved konvertering av {parquet_path}: {e}")
                error_count += 1

    if error_count > 0:
        print(f"Advarsel: {error_count} filer kunne ikke konverteres til GeoParquet")

In [3]:
folder_path = "data/processed"
find_newest_folder(folder_path, use_foldername=True)

('data/processed\\hais_2024-11-05.snappy.parquet', '2024-11-05')

## Strømming

In [24]:
## ----- STRØMMING -----

# Definisjon av FileSystemEventHandler for å håndtere nye filer
class GeoDataHandler(FileSystemEventHandler):
    def __init__(self, incoming_dir, raw_dir, processed_dir, source_dir):
        self.incoming_dir = incoming_dir
        self.raw_dir = raw_dir
        self.processed_dir = processed_dir
        self.source_dir = source_dir

    def on_created(self, event):
        # Vi er kun interessert i filhendelser (ikke mappeopprettelser)
        if not event.is_directory:
            filepath = event.src_path
            filename = os.path.basename(filepath)

            # Sjekk om det er en parquet-fil
            if filename.endswith('.parquet'):
                print(f"Oppdaget ny fil: {filename}")

                # Vent litt for å sikre at filen er ferdig skrevet
                time.sleep(1)

                # Flytt filen til raw-mappen
                destination = os.path.join(self.raw_dir, filename)
                shutil.move(filepath, destination)

                # Prosesser filen
                self.process_file(destination)

    def process_file(self, filepath):
        try:
            # Definer målsti for den konverterte filen
            base_filename = os.path.splitext(os.path.basename(filepath))[0]
            output_path = os.path.join(self.processed_dir, f"{base_filename}")

            # Konverter parquet til GeoParquet med time-partisjonering
            print(f"Konverterer og partisjonerer fil...")
            results = convert_parquet_to_geoparquet(
                filepath,
                output_path,
                partition_columns=["date_time_utc"]  # Vil bruke time-partisjonering
            )

            if results:
                print(f"Konvertering og partisjonering vellykket.")
                # INGEN ARKIVERING TIL SOURCE-MAPPEN HER
            else:
                print(f"Konvertering feilet for {filepath}")

        except Exception as e:
            print(f"Feil ved prosessering av {filepath}: {str(e)}")
            # Logge feilen for senere analyse
            with open(os.path.join(os.path.dirname(self.processed_dir), "error.log"), "a") as log_file:
                timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                log_file.write(f"{timestamp} - {filepath}: {str(e)}\n")

def start_monitoring(data_folder="./data"):
    global _current_observer, global_paths

    # Stopp eksisterende observer hvis den finnes
    if _current_observer is not None:
        _current_observer.stop()
        _current_observer.join()
        print("Stoppet eksisterende overvåking.")

    # Gjenbruk eksisterende mappestruktur eller opprett en ny
    if global_paths is None:
        global_paths = create_folders(data_folder)

    # Opprett event handler med den NYE GeoDataHandler-klassen
    event_handler = GeoDataHandler(
        global_paths["incoming_folder"],
        global_paths["raw_folder"],
        global_paths["processed_folder"],
        global_paths["source_folder"]
    )

    # Start observer
    _current_observer = Observer()
    _current_observer.schedule(event_handler, global_paths["incoming_folder"], recursive=False)
    _current_observer.start()

    print(f"Starter overvåking av mappen: {global_paths['incoming_folder']}")

    return _current_observer

def start_notebook_monitoring():
    print("Starter overvåking...")
    observer = start_monitoring(data_dir)
    print("Observer startet. Kjør stop_monitoring() i en annen celle for å stoppe.")
    return observer

def stop_monitoring():
    global _current_observer
    if _current_observer is not None:
        _current_observer.stop()
        _current_observer.join()
        _current_observer = None

def stop_notebook_monitoring():
    global notebook_observer
    if notebook_observer is not None:
        stop_monitoring()
        notebook_observer = None
        print("Overvåking stoppet.")
    else:
        print("Ingen aktiv overvåking å stoppe.")

def simulate_streaming(data_folder="./data", num_files=5, min_interval=2, max_interval=10):
    global global_paths

    # Gjenbruk eksisterende mappestruktur eller opprett en ny
    if global_paths is None:
        global_paths = create_folders(data_folder)

    source_dir = global_paths["source_folder"]
    incoming_dir = global_paths["incoming_folder"]

    # Finn alle parquet-filer i kildemappen
    files = glob.glob(os.path.join(source_dir, "*.parquet"))

    if not files:
        print(f"Ingen .parquet-filer funnet i {source_dir}")
        return

    print(f"Fant {len(files)} nye filer. Starter simulert strømming...")

    # Begrens antall filer til num_files eller antall tilgjengelige filer
    files_to_process = min(len(files), num_files)

    #for i in range(files_to_process):
    i = 0
    for file in files:
        # Velg en tilfeldig fil
        #file_idx = random.randint(0, len(files)-1)
        #file = files[i]

        # Generer et tilfeldig tidsintervall
        wait_time = random.uniform(min_interval, max_interval)
        time.sleep(wait_time)

        filename = f"{i+1}_{os.path.basename(file)}"  # Legg til indeks for å unngå duplikater
        destination = os.path.join(incoming_dir, filename)
        print(f"Destination {destination}")

        # Kopier filen (ikke flytt, så vi kan bruke den flere ganger)
        shutil.move(file, destination)
        i = i + 1

## Kjør denne for å starte overvåkingen

In [26]:
notebook_observer = start_notebook_monitoring()

Starter overvåking...
Starter overvåking av mappen: C:\Users\esper18\CodeProjects\CN\kaidata_geolake\Christine\data\incoming
Observer startet. Kjør stop_monitoring() i en annen celle for å stoppe.


## Kjør denne funksjonen for å stoppe overvåking

In [25]:
stop_notebook_monitoring()

Overvåking stoppet.


## Denne cellen kan kjøres for å simulere strømming

In [27]:
def start_notebook_simulation():

    # Sjekk om source-mappen inneholder filer
    source_files = glob.glob(os.path.join(global_paths['source_folder'], "*.parquet"))

    simulate_streaming(data_dir, num_files)
    print("Simulering fullført.")

start_notebook_simulation()

Fant 3 nye filer. Starter simulert strømming...
Destination C:\Users\esper18\CodeProjects\CN\kaidata_geolake\Christine\data\incoming\1_hais_2024-10-01.snappy.parquet
Oppdaget ny fil: 1_hais_2024-10-01.snappy.parquet
Konverterer og partisjonerer fil...
Konvertering og partisjonering vellykket.
Destination C:\Users\esper18\CodeProjects\CN\kaidata_geolake\Christine\data\incoming\2_hais_2024-10-24.snappy.parquet
Oppdaget ny fil: 2_hais_2024-10-24.snappy.parquet
Konverterer og partisjonerer fil...
Konvertering og partisjonering vellykket.
Destination C:\Users\esper18\CodeProjects\CN\kaidata_geolake\Christine\data\incoming\3_hais_2024-11-05.snappy.parquet
Oppdaget ny fil: 3_hais_2024-11-05.snappy.parquet
Simulering fullført.
Konverterer og partisjonerer fil...
Konvertering og partisjonering vellykket.
